
# Aprendizaje Profundo
## Self-Supervised Learning Assignment

(This assignment is an adaptation from a proposal originally designed by **Antonio Ríos**).

The goal of this exercise is to perform a simple *Self-supervised learning* experiment. In particular, we are going to implement the method proposed in the RotNet paper that was briefly seen in the theory part, and apply it to the Fashion MNIST dataset.

First, upload this notebook to **google colab** and select the T4 GPU environment. Then, follow the next instructions.

## First part: Supervised training

* Load the ***Fashion MNIST*** dataset using the implemented `tf.keras` loading function.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Cargar el dataset Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step



* Make a division of the training set. The first 10.000 samples will remain as they are with their labels. Store the rest (without labels) in another set. This unlabeled set will be used later in the second part for the self-supervised experiment.

In [ ]:
x_labeled = x_train[:10000]
y_labeled = y_train[:10000]
x_unlabeled = x_train[10000:]



* We set the random seed to a fixed value for reproducibility, and use the following CNN architecture (this is already implemented).

* Complete the following code by compiling this model with the *Categorical Cross-Entropy* loss function, a *SGD* optimizer and a *batch_size* of 512.

* Train the model with **ten epochs** and annotate the accuracy obtained for the test set.

In [ ]:
tf.keras.utils.set_random_seed(1)

model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                            input_shape=[28, 28, 1]),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        tf.keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation="softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

history = model.fit(x_train, y_train, epochs=10, batch_size=512, validation_data=(x_test, y_test))
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print("Test accuracy:", test_acc)


Epoch 1/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 597s 5s/step - accuracy: 0.2573 - loss: 2.9081 - val_accuracy: 0.7289 - val_loss: 0.8884
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 620s 5s/step - accuracy: 0.5910 - loss: 1.2147 - val_accuracy: 0.7801 - val_loss: 0.6540
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 607s 5s/step - accuracy: 0.6735 - loss: 0.9632 - val_accuracy: 0.7983 - val_loss: 0.5576
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 621s 5s/step - accuracy: 0.7209 - loss: 0.8280 - val_accuracy: 0.8042 - val_loss: 0.5393
Epoch 5/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 620s 5s/step - accuracy: 0.7460 - loss: 0.7599 - val_accuracy: 0.8216 - val_loss: 0.4877
Epoch 6/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 623s 5s/step - accuracy: 0.7671 - loss: 0.6929 - val_accuracy: 0.8413 - val_loss: 0.4525
Epoch 7/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 621s 5s/step - accuracy: 0.7802 - loss: 0.6499 - val_accuracy: 0.8512 - val_loss: 0.4263
Epoch 8/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 609s 5s/step - accuracy: 0.7968 - loss: 0.6021 - val_accu

## Second part: Pretext task training

* Implement a function to generate pseudo-labels from the unlabeled set. For this, first you must choose a random number between 0 and 3. According to the obtained random value, apply a rotation (90º, 180º, 270º, No rotation). Each sample will be assigned to its corresponding label.


In [ ]:
import numpy as np

def generate_pseudo_labels(x_unlabeled):
    pseudo_images = []
    pseudo_labels = []
    for img in x_unlabeled:
        r = np.random.randint(0, 4)
        if r == 0:
            rotated_img = img
        elif r == 1:
            rotated_img = np.rot90(img, k=1)
        elif r == 2:
            rotated_img = np.rot90(img, k=2)
        elif r == 3:
            rotated_img = np.rot90(img, k=3)
        pseudo_images.append(rotated_img)
        pseudo_labels.append(r)
    return np.array(pseudo_images), np.array(pseudo_labels)


* Reimplement the same previous convolutional architecture, but adapting the last layer to the label space of our pretext task.

In [ ]:
tf.keras.utils.set_random_seed(1)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 7, activation="relu", padding="same", input_shape=[28, 28, 1]),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    tf.keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation="softmax")
])



* Compile the pretext model with the same parameters than the previous one.

* Train the pretext model on 10 epochs.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

x_pseudo, y_pseudo = generate_pseudo_labels(x_unlabeled)
y_pseudo = tf.keras.utils.to_categorical(y_pseudo, 4)

history = model.fit(x_pseudo, y_pseudo, epochs=10, batch_size=512)


Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 473s 5s/step - accuracy: 0.3586 - loss: 2.4610
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 471s 5s/step - accuracy: 0.6878 - loss: 0.8091
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 472s 5s/step - accuracy: 0.8114 - loss: 0.5277
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 500s 5s/step - accuracy: 0.8620 - loss: 0.3952
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 504s 5s/step - accuracy: 0.8919 - loss: 0.3162
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 471s 5s/step - accuracy: 0.9140 - loss: 0.2614
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 499s 5s/step - accuracy: 0.9238 - loss: 0.2366
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 468s 5s/step - accuracy: 0.9342 - loss: 0.2073
Epoch 9/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 501s 5s/step - accuracy: 0.9400 - loss: 0.1909
Epoch 10/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 506s 5s/step - accuracy: 0.9468 - loss: 0.1696


## Third part: Fine-tuning for the downstream task

* Replace the last layer of the pretrained rotation model by a dense layer to classify the labeled dataset. For this, it is recommended to check the functions `pop()` and `add()` from the `keras` secuential API.

In [ ]:
model.pop()
model.add(tf.keras.layers.Dense(10, activation="softmax"))

* **Freeze** the weights of the *first three convolutional layers*, as in the original paper, in which best embeddings are those from the second convolutional block. You can get the name of a layer using the method _layer.name_. The layers to be freezed should be called something like _conv2d_5, conv2d_6, conv2d_7_.

In [ ]:
for layer in model.layers:
    if layer.name in ["conv2d_5", "conv2d_6", "conv2d_7"]:
        layer.trainable = False


* Compile the model with the same loss and optimizer than the previous ones.

* Train the model using the labeled training set. For this, use the same number of epochs (10) than in the supervised experiment.

In [ ]:
y_labeled = tf.keras.utils.to_categorical(y_labeled, 10)
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
history = model.fit(x_labeled, y_labeled, epochs=10, batch_size=512)



Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.1371 - loss: 2.9915
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.2677 - loss: 1.9980
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.3193 - loss: 1.8378
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.3846 - loss: 1.6744
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.4409 - loss: 1.5560
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.4798 - loss: 1.4397
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.5123 - loss: 1.3583
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.5602 - loss: 1.2489
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.5709 - loss: 1.2196
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.5976 - loss: 1.1331


* Compare the accuracy obtained in the test set between the supervised and the self-supervised approaches, and discuss the results.

Supervised training achieved about 86.5% test accuracy by using explicit labels, which provided clear learning signals.
The loss consistently decreased during training, indicating effective convergence with the labeled data.
In contrast, the self-supervised approach, which does not rely on direct labels, only reached about 59.8% accuracy.
This lower performance highlights the difficulty of learning robust features without explicit guidance.
Overall, the results demonstrate that supervised learning can lead to much better performance when high-quality labeled data is available.